# Importation

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
import random
import os
import pandas as pd
import numpy as np

Le programme suivant permet de réequilibrer les volumes sonore en dB pour ne pas avoir le bruit naturelle trop fort ou trop faible par rapport au son à classifier.

In [ ]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

On import le drive où sont contenue les sons

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Contient le chemiin de la base de données.
PATH = "drive/MyDrive/Projet Cassiopée/Programmes/Dataset/"

In [ ]:
data = pd.read_csv(PATH + "metadata/metadata.csv", sep=";")
data.head(5)

,filename,path,Coup par coups,Explosion,Gros calibre,Petit calibre,Rafale,Chenille,Bateau,Camion,Groupe éléctrogène,Moto,Quad,Voiture,Cris,Discussion calme,Enfant,Femme,Homme
0,Compilation - bruitages _ sons de cris d'enfan...,Voix/Cris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,Compilation - bruitages _ sons de cris d'enfan...,Voix/Cris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,Compilation - bruitages _ sons de cris d'enfan...,Voix/Cris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,Compilation - bruitages _ sons de cris d'enfan...,Voix/Cris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,Compilation - bruitages _ sons de cris d'enfan...,Voix/Cris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


# Augmentation

Cette cellule permet de créer des bruit pour toute les classes dans liste_category, de créer nombre_de_bruit bruits pour chaque son de type nom_bruit. Par exemple ici pour chaque son de char deux bruits de pluie seront créer.

In [ ]:
liste_category = ['Chenille/Donnée']                        # La catégorie de son à modifier
nom_bruit = "rain"                                          # La classe de bruit à créer
nombre_de_bruit = 2                                         # Le nombre de bruit à créer pour chaque fichier son

for category in liste_category:
  liste_doc = np.array(data[['filename']][ data['category'] == category]).flatten()

  for file_name in liste_doc:
    for i in range(nombre_de_bruit):
      sound1 = AudioSegment.from_file(PATH + 'sound/' + category + '/' + file_name)             # Le son original
      sound1 = match_target_amplitude(sound1, -20.0)                                            # On réequilibre les niveau sonore pour que le son original ne soit pas trop fort

      sound2 = AudioSegment.from_file(PATH + 'sound/' + nom_bruit + "/" + random.choice(os.listdir(PATH + "sound/Bruit naturelle" + nom_bruit + "/")))          # Le bruit
      sound2 = match_target_amplitude(sound2, -18)                                                                                                              # On réequilibre les niveau sonore pour que le bruit ne soit pas trop fort

      combined = sound1.overlay(sound2)                                                                                           # On combine les deux bruits
      combined.export(PATH + 'sound/' + category + '/' + file_name[:-4] + nom_bruit + "-" + str(i), format='wav')                 # Le chemin ou doit être exporter le fichier combiné   

Cette cellule fait exactement le même travail que la classe précédent sauf que le type de bruit n'est pas à choisir. Il est directement pris au hasard dan la liste des bruit disponible dans bruit naturelle.

In [ ]:
liste_category = ['Arme/Petit calibre/Donnée']                       # La catégorie de son à modifier
nombre_de_classe = 1

for category in liste_category:
  liste_doc = np.array(data[['filename']][ data['path'] == category]).flatten()

  for file_name in liste_doc:
    for i in range(nombre_de_classe):
      sound1 = AudioSegment.from_file(PATH + 'sound/' + category + '/' + file_name)             # Le son original
      sound1 = match_target_amplitude(sound1, -20.0)                                            # On réequilibre les niveau sonore pour que le son original ne soit pas trop fort

      nom_bruit = random.choice(os.listdir(PATH + "sound/Bruit naturelle/"))                   # Bruit pris au hasard
      sound2 = AudioSegment.from_file(PATH + 'sound/Bruit naturelle/' + nom_bruit + "/" + random.choice(os.listdir(PATH + "sound/Bruit naturelle/" + nom_bruit + "/")))         #Le bruit
      sound2 = match_target_amplitude(sound2, -18)                                                                                                                              # On réequilibre les niveau sonore pour que le bruit ne soit pas trop fort

      combined = sound1.overlay(sound2)                                                                                                                       # On combine les deux bruits
      combined.export(PATH + 'sound/Arme/Petit calibre/Augmentation/' + file_name[:-4] + "-" + nom_bruit + "-" + str(i) + ".wav", format='wav')               # Le chemin ou doit être exporter le fichier combiné   